In [4]:
import pandas as pd
import numpy as np
import os

df1 = pd.read_csv('MLP_embed-86.csv')
df2 = pd.read_csv('output (2).csv')
df1 = df1.sort_values(by=['ID'], ignore_index=True)
df2 = df2.sort_values(by=['ID'], ignore_index=True)
(df1['Target'] == df2['target']).mean()

0.8662704309063893